In [1]:
import yaml
import json
import pe_modules
import dataloaders
import llms

/opt/homebrew/anaconda3/envs/llm-pe/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = config = yaml.safe_load(open("./configs/david_base_config.yaml"))

In [3]:
from item_scorers.ce_scorer import CEScorer

scorer = CEScorer(config)

In [4]:
# Dataloader Class
dataloader_class = dataloaders.DATALOADER_CLASSES[config['data']['data_loader_name']]
# Load item data
item_dataloader = dataloader_class(config['data']['data_path'], config) 

In [5]:
from pe_modules.dt_pe_module import DTPEModule
pe_module = DTPEModule(config, item_dataloader)

In [51]:
# Modify the past aspects
pe_module.aspects = [
                    {'aspect_key': 'age', 'aspect_value': '30 years old'},
                    {'aspect_key': 'fuel efficiency', 'aspect_value': 'guzzler'},
                    {'aspect_key': 'price', 'aspect_value': 'cheap'},
                    ]
# pe_module.aspects = []

In [52]:
# Get aspect for item_desc
item_desc = "Bob's 30 year old truck. As cheap as you get. It's a bit of a guzzler but it wont need to go into the shop much"
aspect_dict = pe_module.get_aspect(item_desc)
print(aspect_dict)

{'aspect_key': 'maintenance', 'aspect_value': 'low'}


## Aspect Generation Results

### Note: Each table is in the order that the aspects key value pairs were generated. E.g. after (year,1968) was generated, that aspect was listed as a previously usd aspect value pair for all subsequent queries with that description. The next pair was (color, cherry red), etc.

Description: 1968 Red Porsche 911. A classic 2-door sports car designed for the enthusiastic collector. Comes equipped with premium leather seats and a beautiful cherry red finish. Visit one of our dealerships to discuss pricing.

| Key | Value |
| --- | ----- |
| year | 1968 |
| color | cherry red |
| body type | 2-door sports car |
| features | premium leather seats |
| purpose | enthusiastic collector |
| dealership | visit one of our dealerships |
| model | Porsche 911 |
| type | classic |
| type | sports car |
| type | classic sports car |

Description: small red men's tshirt

| Key | Value |
| --- | ----- |
| size | small |
| color | red |
| gender | men's |
| style | tshirt |
| type | men's tshirt |

Description: Bob's 30 year old truck. As cheap as you get. It's a bit of a guzzler but it wont need to go into the shop much

| Key | Value |
| --- | ----- |
| age | 30 years old |
| fuel efficiency | guzzler |
| price | cheap |
| maintenance | low |



In [19]:
# Get entailment for aspect_str and various test items
items = {
    "1" : {'description': "Large Red Car"},
    "2" : {'description': "Small Red Car"},
    "3" : {'description': "Large Blue Car"},
    "4" : {'description': "Small Blue Car"},
}
aspect_str = "colour: not red"

like_probs = scorer.score_items(aspect_str, items)
print(like_probs)

{'1': 0.0052513983100652695, '2': 0.01000937819480896, '3': 4.5004278945270926e-05, '4': 5.5663029343122616e-05}
